In [2]:
import cv2 as cv
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import matplotlib.pyplot as plt
from scipy.spatial import distance
import pyautogui as pg
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def get_middle(img, landmark):
    
       
    # results.pose_landmarks 에 감지한 관절의 좌표가 저장되어 있다.
        
    # 왼쪽, 오른쪽 어깨 관절의 중심점 찾기
    x_LEFT_SHOULDER = landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x*img.shape[1]
    y_LEFT_SHOULDER = landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y*img.shape[0]
    z_LEFT_SHOULDER = landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z*100
        
    x_RIGHT_SHOULDER = landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x*img.shape[1]
    y_RIGHT_SHOULDER = landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y*img.shape[0]
    z_RIGHT_SHOULDER = landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z*100
    
    x_MIDDLE_SHOULDER = (x_LEFT_SHOULDER + x_RIGHT_SHOULDER) / 2
    y_MIDDLE_SHOULDER = (y_LEFT_SHOULDER + y_RIGHT_SHOULDER) / 2
    z_MIDDLE_SHOULDER = (z_LEFT_SHOULDER + z_RIGHT_SHOULDER) / 2
    
    MIDDLE_SHOULDER = (int(x_MIDDLE_SHOULDER), int(y_MIDDLE_SHOULDER))
    
    return MIDDLE_SHOULDER

def calculate_angle(img, landmark):
    
    x_LEFT_SHOULDER = landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x*img.shape[1]
    y_LEFT_SHOULDER = landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y*img.shape[0]
    LEFT_SHOULDER = (x_LEFT_SHOULDER, y_LEFT_SHOULDER)
    
    x_RIGHT_SHOULDER = landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x*img.shape[1]
    y_RIGHT_SHOULDER = landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y*img.shape[0]
    RIGHT_SHOULDER = (x_RIGHT_SHOULDER, y_RIGHT_SHOULDER)
    
    
    NOSE = (landmark[mp_pose.PoseLandmark.NOSE].x*img.shape[1],
            landmark[mp_pose.PoseLandmark.NOSE].y*img.shape[0])
    
    ## 코 - 왼쪽 어깨 - 오른쪽 어깨 각도 계산
    vector_1 = [NOSE[i] - LEFT_SHOULDER[i] for i in range(len(NOSE))]    
    vector_2 = [RIGHT_SHOULDER[i] - LEFT_SHOULDER[i] for i in range(len(RIGHT_SHOULDER))] 
    norm1 = np.linalg.norm(vector_1)
    norm2 = np.linalg.norm(vector_2)
        
    # 두 벡터의 내적 계산
    dot_product_1 = np.dot(vector_1, vector_2)
        
    # 두 벡터 사이의 각도 계산 (라디안)
    cos_theta_1 = dot_product_1 / (norm1 * norm2)
    theta_rad_1 = np.arccos(cos_theta_1)
    
    # 라디안을 각도로 변환하여 반환 (도)
    theta_deg_LEFT = np.degrees(theta_rad_1)
    
    
    
    ## 코 - 오른쪽 어깨 - 왼쪽 어깨 각도 계산
    vector_3 = [NOSE[i] - RIGHT_SHOULDER[i] for i in range(len(NOSE))]    
    vector_4 = [LEFT_SHOULDER[i] - RIGHT_SHOULDER[i] for i in range(len(LEFT_SHOULDER))] 
    norm3 = np.linalg.norm(vector_3)
    norm4 = np.linalg.norm(vector_4)
        
    dot_product_2 = np.dot(vector_3, vector_4)
        
    cos_theta_2 = dot_product_2 / (norm3 * norm4)
    theta_rad_2 = np.arccos(cos_theta_2)
    
    theta_deg_RIGHT = np.degrees(theta_rad_2)
        
    return [int(theta_deg_LEFT), int(theta_deg_RIGHT)]



def calculate_distance(img, landmark):
    
    MIDDLE_SHOULDER = get_middle(img, landmark)
    
    NOSE = (landmark[mp_pose.PoseLandmark.NOSE].x*img.shape[1],
            landmark[mp_pose.PoseLandmark.NOSE].y*img.shape[0])
    
    dist = distance.euclidean(NOSE, MIDDLE_SHOULDER)
    
    return int(dist)
    

In [3]:

# 처음 10초 동안의 자세을 통해 사용자의 올바른 자세 기준을 만들기 위한 변수
left_criteria_list = []
right_criteria_list = []
dist_criteria_list = [] 
left_criteria = 0
right_criteria = 0
dist_criteria = 0   
start = time.time()
now = 0
font =  cv.FONT_HERSHEY_PLAIN
message = True

# video feed
cap = cv.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    # with ~ as : 파일 or 함수를 열고 with 내부의 코드가 실행이 끝나면 닫힘 
    # 즉 with 내부에서는 pose를 이용해서 mediapipe 사용가능
    
    while cap.isOpened():
        ret, frame = cap.read() #ret: 비디오가 정상적으로 불러와졌는지
                                #frame : 실제 비디오 정보
                                
        # Detection
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB) # cv는 비디오를 BGR로 받아오고 
                                                     # mediapipe는 RGB로 비디오를 다루기 때문에 변환해야함
        image.flags.writeable = False
        
        results = pose.process(image) # 실제로 관절 감지를 진행하는 부분
        
        
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        
        
        # 랜드마크 추출
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        # 관절 그리기
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,66), thickness=2, circle_radius=2)
                                    )
        # results.pose_landmarks 에 감지한 관절의 좌표가 저장되어 있다.
        
        
        
        # 어깨 사이 중심점 그리기
        cv.circle(image, get_middle(image, landmarks), 6, (0, 255, 255), -1) # 중심은 빨간색 원으로 그림 
        
        ## 처음 20초의 올바른 자세 기준 만들기
        now = time.time()
        
        # 시각화
        NOSE = (int(landmarks[mp_pose.PoseLandmark.NOSE].x*image.shape[1]),
            int(landmarks[mp_pose.PoseLandmark.NOSE].y*image.shape[0]))
        
        cv.line(image, get_middle(image, landmarks), NOSE, (0, 0, 255), 5)
        cv.putText(image, str(round(dist_criteria, 2)), NOSE, font, 2, (0, 255, 0), 3, cv.LINE_AA)
        LEFT = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x*image.shape[1]),
            int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y*image.shape[0]))
        RIGHT = (int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x*image.shape[1]),
            int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y*image.shape[0]))
        
        cv.line(image, NOSE, LEFT, (255, 0, 0), 5)
        cv.line(image, NOSE, RIGHT, (255, 0, 0), 5)
        cv.putText(image, str(round(left_criteria, 2)), LEFT, font, 2, (0, 255, 0), 3, cv.LINE_AA)
        cv.putText(image, str(round(right_criteria, 2)), RIGHT, font, 2, (0, 255, 0), 3, cv.LINE_AA)
        
                
        if now - start <20:
            text = 'Time remaining for setup: ' + str(20 - int((now - start))) + 'sec'
            cv.putText(image, text, (60, 40), font, 2, (255, 255, 255), 3, cv.LINE_AA)
            
            left_angle = calculate_angle(image, landmarks)[0]
            right_angle = calculate_angle(image, landmarks)[1]
            dist = calculate_distance(image, landmarks)  
            
            left_criteria_list.append(left_angle)
            right_criteria_list.append(right_angle)
            dist_criteria_list.append(dist)
            
            left_criteria = np.mean(left_criteria_list) # 알림을 보낼 왼쪽 어깨 각도 기준
            right_criteria = np.mean(right_criteria_list) # 알림을 보낼 오른쪽 어깨 각도 기준
            dist_criteria = np.mean(dist_criteria_list) # 알림을 보낼 코와 어깨 중심 사이의 거리 기준
        
        # 기준 설정이 완료된 후 알림    
        elif message:
            con = pg.confirm('다시 진행하시겠습니까?', '자세 기준 설정이 완료되었습니다.')
            
            if con == 'Cancel':
                pg.alert(text='설정한 기준으로 프로그램을 시작합니다.', title='', button='OK')
                message = False
                
            elif con == 'OK':
                pg.alert(text='자세 기준을 다시 설정합니다..', title='', button='OK')
                start = time.time()
                left_criteria_list = []
                right_criteria_list = []
                dist_criteria_list = [] 
                
        # 기준 설정이 완료 된 후 자세 판단   
        if now - start > 20:
            dist = calculate_distance(image, landmarks)
            angle = calculate_angle(image, landmarks)
        
            if angle[0] > left_criteria*1.3: #왼쪽 어깨 각도가 기준보다 +30% 이상 차이가 나면 알림 
                pg.alert(text='angle_left', title='제목입니다', button='OK')
                
            if angle[1] > right_criteria*1.3: #오른쪽 어깨 각도가 기준보다 +30%이상 차이가 나면 알림
                pg.alert(text='angle_right', title='제목입니다', button='OK')
                
            if dist < dist_criteria*0.8:
                pg.alert(text='dist', title='제목입니다', button='OK') #코와 어깨 중심 사이의 거리가 20%이상 가까워지면 알림
            
                             
        cv.imshow("Tutuleneck_correction", image)

        
        
        if cv.waitKey(1) == ord('q'): # q누르면 break
            break
        
               
    cap.release()
    cv.destroyAllWindows()
    #
    